In [1]:
import pandas as pd
import numpy as np
import requests
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
os.makedirs('data', exist_ok=True)

In [4]:
%%writefile usgs_earthquake_preprocessing.py
"""
USGS Earthquake Data Preprocessing Script
"""

import pandas as pd
import numpy as np
import requests
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


class USGSEarthquakeProcessor:
    """
    Class to process earthquake data from USGS
    """

    def __init__(self, data_source=None):
        self.data_source = data_source
        self.raw_data = None
        self.processed_data = None

    def load_data_from_file(self, file_path):
        """Load data from CSV file"""
        print(f"Reading file: {file_path}")

        try:
            self.raw_data = pd.read_csv(file_path)
            print(f"Successfully loaded {len(self.raw_data)} records")
        except Exception as e:
            print(f"Error loading file: {str(e)}")

    def explore_data(self):
        """Explore initial data"""
        if self.raw_data is None:
            print("No data loaded yet")
            return

        print("\n" + "="*80)
        print("DATA INFORMATION")
        print("="*80)

        print(f"\n Data shape: {self.raw_data.shape}")
        print(f"   - Rows: {self.raw_data.shape[0]}")
        print(f"   - Columns: {self.raw_data.shape[1]}")

        print(f"\n Missing values:")
        missing = self.raw_data.isnull().sum()
        missing_percent = (missing / len(self.raw_data) * 100).round(2)

        missing_df = pd.DataFrame({
            'Column': missing.index,
            'Missing Count': missing.values,
            'Missing %': missing_percent.values
        })
        missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing %', ascending=False)

        if len(missing_df) > 0:
            print(missing_df.to_string(index=False))
        else:
            print("    No missing values")

    def drop_columns(self, columns_to_drop=['dmin', 'magError', 'magNst']):
        """Drop columns with too many missing values"""
        if self.raw_data is None:
            print(" No data to process")
            return

        print("\n" + "="*80)
        print("  DROPPING COLUMNS")
        print("="*80)

        existing_cols = [col for col in columns_to_drop if col in self.raw_data.columns]

        if existing_cols:
            print(f"\n Dropping columns: {existing_cols}")

            for col in existing_cols:
                missing_pct = (self.raw_data[col].isnull().sum() / len(self.raw_data) * 100).round(2)
                print(f"   - {col}: {missing_pct}% missing")

            self.raw_data = self.raw_data.drop(columns=existing_cols)
            print(f"\n Dropped {len(existing_cols)} columns")

    def impute_missing_values(self, columns_to_impute=['nst', 'gap', 'horizontalError'], method='mean'):
        """Impute missing values with mean"""
        if self.raw_data is None:
            print(" No data to process")
            return

        print("\n" + "="*80)
        print(" IMPUTING MISSING VALUES")
        print("="*80)

        for col in columns_to_impute:
            if col in self.raw_data.columns:
                missing_before = self.raw_data[col].isnull().sum()

                if missing_before > 0:
                    if method == 'mean':
                        fill_value = self.raw_data[col].mean()
                    elif method == 'median':
                        fill_value = self.raw_data[col].median()
                    else:
                        fill_value = self.raw_data[col].mode()[0]

                    self.raw_data[col].fillna(fill_value, inplace=True)

                    print(f"\n {col}:")
                    print(f"   - Missing: {missing_before}")
                    print(f"   - Filled with {method}: {fill_value:.4f}")

    def convert_time_format(self, time_column='time'):
        """Convert time format to datetime"""
        if self.raw_data is None:
            print("No data to process")
            return

        print("\n" + "="*80)
        print(" CONVERTING TIME FORMAT")
        print("="*80)

        if time_column not in self.raw_data.columns:
            print(f" Column '{time_column}' not found")
            return

        try:
            print(f"\n Converting column '{time_column}' to datetime...")

            self.raw_data[time_column] = pd.to_datetime(self.raw_data[time_column], utc=True)

            self.raw_data['year'] = self.raw_data[time_column].dt.year
            self.raw_data['month'] = self.raw_data[time_column].dt.month
            self.raw_data['day'] = self.raw_data[time_column].dt.day
            self.raw_data['hour'] = self.raw_data[time_column].dt.hour
            self.raw_data['dayofweek'] = self.raw_data[time_column].dt.dayofweek
            self.raw_data['dayofyear'] = self.raw_data[time_column].dt.dayofyear

            print(f" Successfully converted!")
            print(f"\n Time range:")
            print(f"   - From: {self.raw_data[time_column].min()}")
            print(f"   - To: {self.raw_data[time_column].max()}")

        except Exception as e:
            print(f" Error converting time: {str(e)}")

    def feature_engineering(self):
        """Create additional features"""
        if self.raw_data is None:
            print(" No data to process")
            return

        print("\n" + "="*80)
        print("  FEATURE ENGINEERING")
        print("="*80)

        if 'mag' in self.raw_data.columns:
            print("\n Creating energy column from magnitude...")
            self.raw_data['energy'] = 10 ** (1.5 * self.raw_data['mag'] + 4.8)
            print(" Created 'energy' column")

        if 'mag' in self.raw_data.columns:
            print("\n Categorizing earthquakes by magnitude...")

            def categorize_magnitude(mag):
                if pd.isna(mag):
                    return 'Unknown'
                elif mag < 5.0:
                    return 'Small'
                elif 5.0 <= mag < 6.0:
                    return 'Moderate'
                elif 6.0 <= mag < 7.0:
                    return 'Strong'
                elif 7.0 <= mag < 8.0:
                    return 'Major'
                else:
                    return 'Great'

            self.raw_data['magnitude_category'] = self.raw_data['mag'].apply(categorize_magnitude)
            print(" Created 'magnitude_category' column")

        if 'depth' in self.raw_data.columns:
            print("\n Categorizing earthquake depth...")

            def categorize_depth(depth):
                if pd.isna(depth):
                    return 'Unknown'
                elif depth < 70:
                    return 'Shallow'
                elif 70 <= depth < 300:
                    return 'Intermediate'
                else:
                    return 'Deep'

            self.raw_data['depth_category'] = self.raw_data['depth'].apply(categorize_depth)
            print(" Created 'depth_category' column")

    def remove_duplicates(self):
        """Remove duplicate records"""
        if self.raw_data is None:
            print(" No data to process")
            return

        print("\n" + "="*80)
        print(" REMOVING DUPLICATES")
        print("="*80)

        initial_count = len(self.raw_data)
        self.raw_data = self.raw_data.drop_duplicates()
        final_count = len(self.raw_data)
        duplicates_removed = initial_count - final_count

        print(f"\n Results:")
        print(f"   - Initial records: {initial_count}")
        print(f"   - Final records: {final_count}")
        print(f"   - Duplicates removed: {duplicates_removed}")

    def standardize_columns(self):
        """Standardize column names"""
        if self.raw_data is None:
            print(" No data to process")
            return

        print("\n" + "="*80)
        print(" STANDARDIZING COLUMN NAMES")
        print("="*80)

        column_mapping = {
            'time': 'Timestamp',
            'mag': 'Magnitude',
            'latitude': 'Latitude',
            'longitude': 'Longitude',
            'depth': 'Depth'
        }

        existing_mapping = {k: v for k, v in column_mapping.items() if k in self.raw_data.columns}

        if existing_mapping:
            self.raw_data = self.raw_data.rename(columns=existing_mapping)
            print(" Standardized column names:")
            for old, new in existing_mapping.items():
                print(f"   - {old} → {new}")

    def save_processed_data(self, output_path='processed_earthquake_data.csv'):
        """Save processed data"""
        if self.raw_data is None:
            print(" No data to save")
            return

        print("\n" + "="*80)
        print("SAVING DATA")
        print("="*80)

        try:
            self.processed_data = self.raw_data.copy()
            self.processed_data.to_csv(output_path, index=False)

            print(f"\n Data saved successfully!")
            print(f" File: {output_path}")
            print(f" Records: {len(self.processed_data)}")
            print(f" Columns: {len(self.processed_data.columns)}")

        except Exception as e:
            print(f" Error saving file: {str(e)}")

Writing usgs_earthquake_preprocessing.py


In [ ]:
from google.colab import files

print("Please select your earthquake data files...")
print("Select all 4 CSV files:")
print("  - usgs_earthquake_data_2002_2011.csv")
print("  - usgs_earthquake_data_2011_2013.csv")
print("  - usgs_earthquake_data_2013_2015.csv")
print("  - usgs_earthquake_data_2015_2025.csv")
print("\nClick 'Choose Files' button below ")

uploaded = files.upload()

# Move files to data directory
for filename in uploaded.keys():
    os.rename(filename, f'data/{filename}')
    print(f" Uploaded: {filename}")

print("\n All files uploaded successfully!")

Please select your earthquake data files...
Select all 4 CSV files:
  - usgs_earthquake_data_2002_2011.csv
  - usgs_earthquake_data_2011_2013.csv
  - usgs_earthquake_data_2013_2015.csv
  - usgs_earthquake_data_2015_2025.csv

Click 'Choose Files' button below 


TypeError: 'NoneType' object is not subscriptable

In [ ]:
print("Files in data directory:")
!ls -lh data/

Files in data directory:
total 547M
-rw-r--r-- 1 root root 181M Dec  8 14:07 usgs_earthquake_data_2002_2011.csv
-rw-r--r-- 1 root root  47M Dec  8 14:06 usgs_earthquake_data_2011_2013.csv
-rw-r--r-- 1 root root  48M Dec  8 14:06 usgs_earthquake_data_2013_2015.csv
-rw-r--r-- 1 root root 273M Dec  8 14:08 usgs_earthquake_data_2015_2025.csv


In [ ]:
from usgs_earthquake_preprocessing import USGSEarthquakeProcessor

print("="*80)
print("PROCESSING FILE: 2002-2011 DATA")
print("="*80)

# Create processor
processor1 = USGSEarthquakeProcessor()

# Load data
processor1.load_data_from_file('data/usgs_earthquake_data_2002_2011.csv')

# Process
processor1.drop_columns(['dmin', 'magError', 'magNst'])
processor1.impute_missing_values(['nst', 'gap', 'horizontalError'], method='mean')
processor1.convert_time_format('time')
processor1.feature_engineering()
processor1.remove_duplicates()
processor1.standardize_columns()

# Save
processor1.save_processed_data('processed_2002_2011.csv')

print("\n FILE COMPLETE!")

PROCESSING FILE: 2002-2011 DATA
Reading file: data/usgs_earthquake_data_2002_2011.csv
Successfully loaded 1026123 records

  DROPPING COLUMNS

 Dropping columns: ['dmin', 'magError', 'magNst']
   - dmin: 49.35% missing
   - magError: 59.96% missing
   - magNst: 41.48% missing

 Dropped 3 columns

🔧 IMPUTING MISSING VALUES

 nst:
   - Missing: 227423
   - Filled with mean: 20.3557

 gap:
   - Missing: 256403
   - Filled with mean: 136.1259

 horizontalError:
   - Missing: 542422
   - Filled with mean: 0.7536

 CONVERTING TIME FORMAT

 Converting column 'time' to datetime...
 Successfully converted!

 Time range:
   - From: 2002-01-01 00:25:29.220000+00:00
   - To: 2011-04-13 23:58:07.650000+00:00

  FEATURE ENGINEERING

 Creating energy column from magnitude...
 Created 'energy' column

 Categorizing earthquakes by magnitude...
 Created 'magnitude_category' column

🌊 Categorizing earthquake depth...
 Created 'depth_category' column

 REMOVING DUPLICATES

 Results:
   - Initial records: 

In [ ]:
from usgs_earthquake_preprocessing import USGSEarthquakeProcessor

print("="*80)
print("PROCESSING FILE 2/4: 2011-2013 DATA")
print("="*80)

processor2 = USGSEarthquakeProcessor()
processor2.load_data_from_file('data/usgs_earthquake_data_2011_2013.csv')
processor2.drop_columns(['dmin', 'magError', 'magNst'])
processor2.impute_missing_values(['nst', 'gap', 'horizontalError'], method='mean')
processor2.convert_time_format('time')
processor2.feature_engineering()
processor2.remove_duplicates()
processor2.standardize_columns()
processor2.save_processed_data('processed_2011_2013.csv')

print("\n FILE 2/4 COMPLETE!")

PROCESSING FILE 2/4: 2011-2013 DATA
Reading file: data/usgs_earthquake_data_2011_2013.csv
Successfully loaded 257916 records

  DROPPING COLUMNS

 Dropping columns: ['dmin', 'magError', 'magNst']
   - dmin: 39.41% missing
   - magError: 45.04% missing
   - magNst: 29.69% missing

 Dropped 3 columns

🔧 IMPUTING MISSING VALUES

 nst:
   - Missing: 57623
   - Filled with mean: 25.3741

 gap:
   - Missing: 57537
   - Filled with mean: 127.0790

 horizontalError:
   - Missing: 122446
   - Filled with mean: 0.7590

 CONVERTING TIME FORMAT

 Converting column 'time' to datetime...
 Successfully converted!

 Time range:
   - From: 2011-04-14 00:05:44.766000+00:00
   - To: 2013-08-30 23:58:39.575000+00:00

  FEATURE ENGINEERING

 Creating energy column from magnitude...
 Created 'energy' column

 Categorizing earthquakes by magnitude...
 Created 'magnitude_category' column

🌊 Categorizing earthquake depth...
 Created 'depth_category' column

 REMOVING DUPLICATES

 Results:
   - Initial records:

In [ ]:
from usgs_earthquake_preprocessing import USGSEarthquakeProcessor

print("="*80)
print("PROCESSING FILE 3/4: 2013-2015 DATA")
print("="*80)

processor3 = USGSEarthquakeProcessor()
processor3.load_data_from_file('data/usgs_earthquake_data_2013_2015.csv')
processor3.drop_columns(['dmin', 'magError', 'magNst'])
processor3.impute_missing_values(['nst', 'gap', 'horizontalError'], method='mean')
processor3.convert_time_format('time')
processor3.feature_engineering()
processor3.remove_duplicates()
processor3.standardize_columns()
processor3.save_processed_data('processed_2013_2015.csv')

print("\n FILE 3/4 COMPLETE!")

PROCESSING FILE 3/4: 2013-2015 DATA
Reading file: data/usgs_earthquake_data_2013_2015.csv
Successfully loaded 265725 records

  DROPPING COLUMNS

 Dropping columns: ['dmin', 'magError', 'magNst']
   - dmin: 33.16% missing
   - magError: 32.93% missing
   - magNst: 31.1% missing

 Dropped 3 columns

🔧 IMPUTING MISSING VALUES

 nst:
   - Missing: 112354
   - Filled with mean: 16.5288

 gap:
   - Missing: 75805
   - Filled with mean: 127.0553

 horizontalError:
   - Missing: 114429
   - Filled with mean: 1.7898

 CONVERTING TIME FORMAT

 Converting column 'time' to datetime...
 Successfully converted!

 Time range:
   - From: 2013-08-31 00:02:26.723000+00:00
   - To: 2015-08-28 23:48:40.730000+00:00

  FEATURE ENGINEERING

 Creating energy column from magnitude...
 Created 'energy' column

 Categorizing earthquakes by magnitude...
 Created 'magnitude_category' column

🌊 Categorizing earthquake depth...
 Created 'depth_category' column

 REMOVING DUPLICATES

 Results:
   - Initial records:

In [ ]:
from usgs_earthquake_preprocessing import USGSEarthquakeProcessor

print("="*80)
print("PROCESSING FILE 4/4: 2015-2025 DATA")
print("="*80)

processor4 = USGSEarthquakeProcessor()
processor4.load_data_from_file('data/usgs_earthquake_data_2015_2025.csv')
processor4.drop_columns(['dmin', 'magError', 'magNst'])
processor4.impute_missing_values(['nst', 'gap', 'horizontalError'], method='mean')
processor4.convert_time_format('time')
processor4.feature_engineering()
processor4.remove_duplicates()
processor4.standardize_columns()
processor4.save_processed_data('processed_2015_2025.csv')

print("\n FILE 4/4 COMPLETE!")

PROCESSING FILE 4/4: 2015-2025 DATA
Reading file: data/usgs_earthquake_data_2015_2025.csv
Successfully loaded 1489681 records

  DROPPING COLUMNS

 Dropping columns: ['dmin', 'magError', 'magNst']
   - dmin: 31.95% missing
   - magError: 29.11% missing
   - magNst: 27.45% missing

 Dropped 3 columns

🔧 IMPUTING MISSING VALUES

 nst:
   - Missing: 513785
   - Filled with mean: 21.1209

 gap:
   - Missing: 389700
   - Filled with mean: 121.6653

 horizontalError:
   - Missing: 522286
   - Filled with mean: 2.1258

 CONVERTING TIME FORMAT

 Converting column 'time' to datetime...
 Successfully converted!

 Time range:
   - From: 2015-08-01 00:07:41.692000+00:00
   - To: 2025-12-06 09:40:47.601000+00:00

  FEATURE ENGINEERING

 Creating energy column from magnitude...
 Created 'energy' column

 Categorizing earthquakes by magnitude...
 Created 'magnitude_category' column

🌊 Categorizing earthquake depth...
 Created 'depth_category' column

 REMOVING DUPLICATES

 Results:
   - Initial recor

In [ ]:
"""
Merge All Processed Earthquake Data Files
Combines the 4 processed CSV files into one complete dataset
"""

import pandas as pd
import os

def merge_processed_files():
    """
    Merge all processed earthquake data files into one
    """
    print("="*80)
    print(" MERGING PROCESSED EARTHQUAKE DATA FILES")
    print("="*80)

    # Define the processed files to merge
    processed_files = [
        'processed_2002_2011.csv',
        'processed_2011_2013.csv',
        'processed_2013_2015.csv',
        'processed_2015_2025.csv'
    ]

    # List to store all dataframes
    all_dataframes = []

    print("\n STEP 1: Loading processed files...")
    print("-" * 80)

    # Load each file
    for i, filename in enumerate(processed_files, 1):
        if os.path.exists(filename):
            print(f"\n[{i}/4] Loading: {filename}")
            try:
                df = pd.read_csv(filename)
                print(f"    Loaded {len(df):,} records")
                print(f"    Columns: {len(df.columns)}")

                # Show date range (use flexible parsing)
                if 'Timestamp' in df.columns:
                    try:
                        df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='mixed', utc=True)
                        print(f"    Date range: {df['Timestamp'].min()} to {df['Timestamp'].max()}")
                    except:
                        print(f"    Could not parse timestamps for date range")

                all_dataframes.append(df)
            except Exception as e:
                print(f"    Error loading file: {str(e)}")
        else:
            print(f"\n[{i}/4]  File not found: {filename}")

    if not all_dataframes:
        print("\n No files were loaded. Cannot merge.")
        return

    # Check if all files have the same columns
    print("\n" + "="*80)
    print(" STEP 2: Verifying column compatibility...")
    print("-" * 80)

    first_columns = set(all_dataframes[0].columns)
    all_compatible = True

    for i, df in enumerate(all_dataframes[1:], start=2):
        current_columns = set(df.columns)
        if current_columns != first_columns:
            print(f"\n File {i} has different columns:")
            missing = first_columns - current_columns
            extra = current_columns - first_columns
            if missing:
                print(f"   Missing: {missing}")
            if extra:
                print(f"   Extra: {extra}")
            all_compatible = False

    if all_compatible:
        print("\n All files have compatible columns!")
    else:
        print("\n Warning: Files have different columns. Proceeding with merge...")

    # Merge all dataframes
    print("\n" + "="*80)
    print(" STEP 3: Merging dataframes...")
    print("-" * 80)

    merged_df = pd.concat(all_dataframes, ignore_index=True)
    print(f"\n Merged successfully!")
    print(f"   Total records: {len(merged_df):,}")
    print(f"   Total columns: {len(merged_df.columns)}")

    # Check for overlapping data
    print("\n" + "="*80)
    print(" STEP 4: Checking for duplicates...")
    print("-" * 80)

    initial_count = len(merged_df)
    merged_df = merged_df.drop_duplicates()
    final_count = len(merged_df)
    duplicates = initial_count - final_count

    print(f"\n   Initial records: {initial_count:,}")
    print(f"   After removing duplicates: {final_count:,}")
    print(f"   Duplicates removed: {duplicates:,}")

    if duplicates > 0:
        print(f"\n Found {duplicates:,} duplicate records (likely from overlapping date ranges)")
        print("   These have been removed from the merged file.")
    else:
        print("\n No duplicate records found!")

    # Sort by timestamp
    print("\n" + "="*80)
    print(" STEP 5: Sorting by timestamp...")
    print("-" * 80)

    if 'Timestamp' in merged_df.columns:
        try:
            # Ensure timestamp is in datetime format
            if merged_df['Timestamp'].dtype == 'object':
                merged_df['Timestamp'] = pd.to_datetime(merged_df['Timestamp'], format='mixed', utc=True)

            merged_df = merged_df.sort_values('Timestamp').reset_index(drop=True)
            print(f"\n Sorted by Timestamp")
            print(f"   Date range: {merged_df['Timestamp'].min()} to {merged_df['Timestamp'].max()}")
        except Exception as e:
            print(f"\n Could not sort by timestamp: {str(e)}")

    # Display statistics
    print("\n" + "="*80)
    print(" STEP 6: Merged data statistics...")
    print("-" * 80)

    if 'Magnitude' in merged_df.columns:
        print(f"\n Magnitude Statistics:")
        print(f"   Min: {merged_df['Magnitude'].min():.2f}")
        print(f"   Max: {merged_df['Magnitude'].max():.2f}")
        print(f"   Mean: {merged_df['Magnitude'].mean():.2f}")
        print(f"   Median: {merged_df['Magnitude'].median():.2f}")

    if 'magnitude_category' in merged_df.columns:
        print(f"\n Magnitude Distribution:")
        mag_dist = merged_df['magnitude_category'].value_counts().sort_index()
        for category, count in mag_dist.items():
            pct = (count / len(merged_df) * 100)
            print(f"   {category:12s}: {count:8,} ({pct:5.2f}%)")

    if 'depth_category' in merged_df.columns:
        print(f"\n Depth Distribution:")
        depth_dist = merged_df['depth_category'].value_counts().sort_index()
        for category, count in depth_dist.items():
            pct = (count / len(merged_df) * 100)
            print(f"   {category:12s}: {count:8,} ({pct:5.2f}%)")

    if 'year' in merged_df.columns:
        print(f"\n Yearly Distribution:")
        yearly_dist = merged_df['year'].value_counts().sort_index()
        for year, count in yearly_dist.head(5).items():
            print(f"   {year}: {count:7,} earthquakes")
        if len(yearly_dist) > 5:
            print(f"   ... ({len(yearly_dist) - 5} more years)")
            for year, count in yearly_dist.tail(5).items():
                print(f"   {year}: {count:7,} earthquakes")

    # Save merged file
    print("\n" + "="*80)
    print(" STEP 7: Saving merged file...")
    print("-" * 80)

    output_file = 'merged_earthquake_data_2002_2025.csv'

    try:
        merged_df.to_csv(output_file, index=False)

        # Get file size
        file_size = os.path.getsize(output_file)
        size_mb = file_size / (1024 * 1024)

        print(f"\n Merged file saved successfully!")
        print(f"   File: {output_file}")
        print(f"   Records: {len(merged_df):,}")
        print(f"   Columns: {len(merged_df.columns)}")
        print(f"   Size: {size_mb:.2f} MB")

    except Exception as e:
        print(f"\n Error saving file: {str(e)}")
        return

    # Final summary
    print("\n" + "="*80)
    print(" MERGE COMPLETE!")
    print("="*80)

    print(f"\n Summary:")
    print(f"   Input files: {len(processed_files)}")
    print(f"   Successfully loaded: {len(all_dataframes)}")
    print(f"   Total records (before dedup): {initial_count:,}")
    print(f"   Total records (after dedup): {final_count:,}")
    print(f"   Duplicates removed: {duplicates:,}")
    print(f"   Output file: {output_file}")
    print(f"   File size: {size_mb:.2f} MB")
    print("="*80 + "\n")

if __name__ == "__main__":
    merge_processed_files()


 MERGING PROCESSED EARTHQUAKE DATA FILES

 STEP 1: Loading processed files...
--------------------------------------------------------------------------------

[1/4] Loading: processed_2002_2011.csv
    Loaded 1,026,123 records
    Columns: 28
    Date range: 2002-01-01 00:25:29.220000+00:00 to 2011-04-13 23:58:07.650000+00:00

[2/4] Loading: processed_2011_2013.csv
    Loaded 257,916 records
    Columns: 28
    Date range: 2011-04-14 00:05:44.766000+00:00 to 2013-08-30 23:58:39.575000+00:00

[3/4] Loading: processed_2013_2015.csv
    Loaded 265,725 records
    Columns: 28
    Date range: 2013-08-31 00:02:26.723000+00:00 to 2015-08-28 23:48:40.730000+00:00

[4/4] Loading: processed_2015_2025.csv
    Loaded 1,489,681 records
    Columns: 28
    Date range: 2015-08-01 00:07:41.692000+00:00 to 2025-12-06 09:40:47.601000+00:00

 STEP 2: Verifying column compatibility...
--------------------------------------------------------------------------------

 All files have compatible columns!

 S

In [ ]:
import pandas as pd
import os # Import os module to check file existence

file_path = 'merged_earthquake_data_2002_2025.csv'

print("Loading merged earthquake data...")

if not os.path.exists(file_path):
    print(f"Error: The file '{file_path}' was not found.")
    print("Please ensure that the previous cell (merging processed files) was run successfully.")
    print("You might need to re-run the cell that generates 'merged_earthquake_data_2002_2025.csv'.")
else:
    df = pd.read_csv(file_path)

    print(f"Loaded {len(df):,} records with {len(df.columns)} columns\n")

    print("="*80)
    print("MISSING DATA ANALYSIS")
    print("="*80)

    missing_count = df.isnull().sum()
    missing_percent = (missing_count / len(df) * 100).round(2)

    missing_df = pd.DataFrame({
        'Column': missing_count.index,
        'Missing Count': missing_count.values,
        'Missing %': missing_percent.values
    })

    missing_df = missing_df.sort_values('Missing %', ascending=False)

    print("\nColumns with Missing Data:\n")
    cols_with_missing = missing_df[missing_df['Missing Count'] > 0]
    print(cols_with_missing.to_string(index=False))

    total_cells = len(df) * len(df.columns)
    total_missing = missing_count.sum()
    total_missing_pct = (total_missing / total_cells * 100).round(2)

    print("\n" + "="*80)
    print("OVERALL STATISTICS")
    print("="*80)
    print(f"Total rows: {len(df):,}")
    print(f"Total columns: {len(df.columns)}")
    print(f"Total cells: {total_cells:,}")
    print(f"Total missing: {total_missing:,} ({total_missing_pct}%)")
    print(f"Data completeness: {100 - total_missing_pct:.2f}%")
    print(f"\nColumns with missing data: {len(cols_with_missing)}/{len(df.columns)}")
    print(f"Columns with NO missing data: {len(df.columns) - len(cols_with_missing)}/{len(df.columns)}")

Loading merged earthquake data...
Error: The file 'merged_earthquake_data_2002_2025.csv' was not found.
Please ensure that the previous cell (merging processed files) was run successfully.
You might need to re-run the cell that generates 'merged_earthquake_data_2002_2025.csv'.


In [ ]:
import os

file_name = 'merged_earthquake_data_2002_2025.csv'

if os.path.exists(file_name):
    print(f"File '{file_name}' exists.")
    file_size_bytes = os.path.getsize(file_name)
    file_size_mb = file_size_bytes / (1024 * 1024)
    print(f"File size: {file_size_mb:.2f} MB")
else:
    print(f"File '{file_name}' does NOT exist.")
    print("Please ensure the merge process in cell H1Q36L7TWTnV was completed successfully.")

File 'merged_earthquake_data_2002_2025.csv' does NOT exist.
Please ensure the merge process in cell H1Q36L7TWTnV was completed successfully.
